# Simple example of time domain beamforming

Let's imagine that we have one or more idealised point sound sources with spherical spreading impinging upon an n-element linear microphone array.

The situation is depicted in the figure below.

![Linear array](figures/linear_array.png)

Note that there are three different frames of reference here. 

There is the absolute frame of reference where North is $0^o$.
There is the frame of reference relative to the forehead end-fire of the linear array.
For both these frames of reference, the origin is the centre of the array, and bearings
are positive in the clockwise direction, and negative in the anti-clockwise direction.
We use θ to denote the angle of rotation of the array, relative to absolute North.
There is also the standard cartesian coordinate system.
The bearing, $\Omega$, of a point given in cartesian coordinates is
$$ \Omega = atan2(x,y) $$
(Note the order of the elements)

A relative bearing, $phi$, is given by $\Omega - \theta$.

Note that we are using point sources, not the plane-wave approximation.

We have used the following helpful links in developing the code for this example:

* [Time delay estimation for passive sonar signal processing](https://ieeexplore.ieee.org/document/1163560)
* [Jupyter User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html)
* https://en.wikipedia.org/wiki/Wavelength
* [Equations for Plane Waves, Spherical Waves, and Gaussian Beams](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118939154.app3)
* https://github.com/bokeh/bokeh/blob/1.3.2/examples/howto/layouts/dashboard.py
* [Beamforming in the Time Domain](https://onlinelibrary.wiley.com/doi/10.1002/9781119293132.ch11)


For a linear array, a delay of 0 implies a beam-steer direction of broadside (perpendicular
to the array).

A delay equivalent to the spacing between two adjacent elements divided by the velocity of
the soundwave implies a beam steering direction of either endfire (parallel to the array),
where the endfire (front or back) is determined by the sign of the delay

More generally, a delay of $\delta = \cos\phi \frac{d}{c}$ corresponds to a relative beam-steer at
an angle $\phi$ for $\phi \in [0^o, 180^o]$.

Given that we are dealing with discrete time signals, the delay cannot be chosen arbitrarily.

Instead, for sample rate $S$, we have

$$ \cos\phi = \frac{c}{d} \delta = \frac{c}{d} \frac{m}{S} $$

where

$$ -\frac{Sd}{c} < m < \frac{Sd}{c} $$

Note that a linear array cannot differentiate between a negative and a positive relative bearing (left-right ambiguity).


## Import modules

In [ ]:
from ipywidgets import interactive, fixed, GridspecLayout, Layout
from IPython.display import display
import numpy as np
import seaborn as sns
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label, Span, Legend, LegendItem, Range1d
from bokeh.layouts import grid, column, gridplot
from bokeh.models import CustomJS, Slider, ColumnDataSource
import soundfile
from os import path

output_notebook()

## Set up parameters

In order to create interactive Bokeh widgets, it is necessary to separate the plotting and the 
calculating code, so that an update function can be written, which simple updates the data in
the plots.

See:
* https://github.com/bokeh/bokeh/blob/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

We create a class whose single instance will hold all the information regarding this example.

In [ ]:
class Info(object):
    '''Holder for global information'''
    
    def __repr__(self):
        ''' Print a list of the attributes of an instance of the class'''
        print_str = '['
        for attr, value in self.__dict__.items():
            print_str += f"'{attr}',"
        print_str = print_str[:-1] + ']'
        return print_str
    
    def __str__(self):
        ''' Print a list of the attributes and their values for an
        instance of the class'''
        print_str = ''
        for attr, value in self.__dict__.items():
            print_str += f' {attr} : {value}\n'
        return print_str
    
info = Info()

def bearing2cart(r,theta):
    # convert absolute bearing to polar coordinates
    return (r*np.sin(np.deg2rad(theta)), r*np.cos(np.deg2rad(theta)))

def cart2bearing(x,y):
    return ( np.hypot(x,y), np.rad2deg(np.atan2(x,y)) )


In [ ]:
def calculate_everything(X,# Instance of the `Info` class
        # Parameters we wish to vary:
                         num_elements=6,
                         array_angle=0,
                         num_sources=1,
                         src_radius=2,
                         wavelength=0.2,
                         sample_length=0.002,
                         t_0=0,
        # Debug parameters:
                         verbose=False,
                         ):
    '''This function calculates the temporal and spacial solution
    for a simple (idealistic) sound-field when we have a number of
    sound sources and a linear array of receivers.'''
    
    #TODO: make src_radius, wavelength, src_angle, src_amp into lists of
    # length num_sources

#    pass
#if True:

    #-------------------------------------------------------------------
    # Array characteristics
    #-------------------------------------------------------------------

    X.max_elements = 12
    X.array_angle = array_angle
    X.spacing = 0.1 # metres
    X.x_location = 0 # metres
    Arr = []
    X.array_length = X.spacing*(num_elements-1)
    for k in range(num_elements):
        x_pos = (X.array_length/2 - 
                  k*X.spacing)*np.sin(np.deg2rad(array_angle))
        y_pos = (X.array_length/2 - 
                 k*X.spacing)*np.cos(np.deg2rad(array_angle))
        Arr.append([x_pos,y_pos])
    #Arr = np.array(Arr)
    X.Arr = Arr
    X.col = sns.color_palette(None, 2*X.max_elements).as_hex()

    #-------------------------------------------------------------------
    # Source characteristics
    #-------------------------------------------------------------------

    amp_src = 1
    delta_angle = 90
    Src = {}
    Src['position'] = []
    Src['absolute_direction'] = []
    Src['amplitude'] = []
    Src['wavelength'] = []
    for n in range(num_sources):
        src_angle = (n+1)*delta_angle
        x, y = bearing2cart(src_radius, src_angle)
        Src['position'].append([x,y]) # Source position
        Src['absolute_direction'].append(src_angle)
        Src['amplitude'].append(amp_src) # Source amplitude
        Src['wavelength'].append(wavelength) # Source wavelength
    X.Src = Src

    π = np.pi
    c = 340 # m/s (Speed of sound)
    X.c = c

    #-------------------------------------------------------------------
    # Spacial characteristics
    #-------------------------------------------------------------------

    N = 500
    X.x_min = -1
    X.x_max = 1
    x = np.linspace(X.x_min, X.x_max, N)
    y = np.linspace(X.x_min, X.x_max, N)    
    xx, yy = np.meshgrid(x, y)
    
    #-------------------------------------------------------------------
    # Temporal characteristics
    #-------------------------------------------------------------------

    X.sample_rate = 96000
    X.sample_length = sample_length

    # Discrete beam-steer deltas directions
    m = np.arange(int(-np.floor(X.spacing*X.sample_rate/c)),
                  int(np.ceil(X.spacing*X.sample_rate/c)),1)
    
    # These deltas are the same for (-180,0) and (0,180), so for
    # convenience, we duplicate this list of discrete directions
    X.m = np.concatenate([m,np.flip(m)])

    cos_phi = c/X.spacing*X.m/X.sample_rate
    X.phi = np.rad2deg(np.arccos(cos_phi))
    
    # We need to change the sign of the first half of the phi array
    X.phi[:len(X.phi)//2] = - X.phi[:len(X.phi)//2]
    
    X.beam_widths = np.stack([
        np.concatenate((-180,X.phi[:-1] + np.diff(X.phi)/2), axis=None),
        np.concatenate((X.phi[:-1] + np.diff(X.phi)/2,180), axis=None)
    ])
        
    # We need to get enough time samples, so that we can shift
    # by num_elements * (m[-1] - m[0]) without wrapping around
    t = np.arange(t_0 + m[0]*num_elements/X.sample_rate,
            t_0 + m[-1]*num_elements/X.sample_rate + sample_length,
            1/X.sample_rate)
    X.t_0 = t_0
    X.t = t

    #-------------------------------------------------------------------
    # Sound-pressure level over all space at time, t_0
    #-------------------------------------------------------------------
    
    Z = 0*xx
    for n in range(num_sources):
        [s_x, s_y] = Src['position'][n] #location of source
        A = Src['amplitude'][n]
        λ = Src['wavelength'][n]
        f = c/λ # Hz
        rr = np.hypot(xx-s_x,yy-s_y)
        Z = Z + A/rr*np.sin(2*π/λ*(rr - c*t_0))
        if verbose:
            print(f'Source {n+1}: A = {A}, ',
                  f'f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 
    X.Z = Z

    #-------------------------------------------------------------------
    # Sound-pressure level at each array element over all time, t
    #-------------------------------------------------------------------

    Y = []
    for k in range(num_elements):
        F = 0*t
        for n in range(num_sources):
            [s_x, s_y] = Src['position'][n] #location of source
            A = Src['amplitude'][n]
            λ = Src['wavelength'][n] # metres
            r = np.hypot(Arr[k][0]-s_x,Arr[k][1]-s_y)
            F = F + A/r*np.sin(2*π/λ*(r - c*t))
        Y.append(F)
    X.Y = Y
    
    #-------------------------------------------------------------------
    # Sound-pressure level at each array element over all time, t
    # for real wav data
    #-------------------------------------------------------------------

    if num_elements == 2:
        wav_file = 'Data/Pi4sineTest_20210401_101202_trimmed_pure.wav'
    else:
        wav_file = 'no_file'
    print(wav_file)
    if path.isfile(wav_file):
        real_data = True
        raw_wav, sr = soundfile.read(wav_file) 
        W = [raw_wav[:len(info.t),0],raw_wav[:len(info.t),1]]
    else:
        real_data = False
        W = []
        for k in range(num_elements):
            W.append(0*t)
    
    X.real_data = real_data
    X.W = W
        
    
    #-------------------------------------------------------------------
    # Time-domain beamforming
    #-------------------------------------------------------------------

    a = np.argmin(np.abs(t-t_0))
    b = np.argmin(np.abs(t-t_0-sample_length))
    # We create a delay-sum time series for each look direction by
    # shifting the time-series for each element by an amount, m*k,
    # padding with zeros
    beams = []
    beams_real = []
    sum_F = []
    sum_F_real = []
    
    for j in X.m:
        current_beam = 0*Y[0]
        current_beam_real = 0*Y[0]        
        for k in range(num_elements):
            current_beam = (current_beam + pad(Y[k],-j*k))
            current_beam_real = (current_beam_real + pad(W[k],-j*k))
        current_beam = current_beam/num_elements
        current_beam_real = current_beam_real/num_elements
        sum_F.append(current_beam)
        sum_F_real.append(current_beam_real)
        # We only sum the section of the beam which doesn't contain
        # any padding
        beams.append(np.sum(np.abs(current_beam[a:b]))/(b-a))
        beams_real.append(np.sum(np.abs(current_beam_real[a:b]))/(b-a))
    X.sum_F = sum_F
    X.beams = beams
    X.sum_F_real = sum_F_real
    X.beams_real = beams_real

    return X

def pad(F,n):
    '''Return an array of the same size as F, where the elements of
    F are shifted by n, padding with zeros.'''
    if n == 0:
        return F
    if n > 0:
        return np.concatenate([np.zeros(n), F[:-n]])
    if n < 0:
        return np.concatenate([F[-n:], np.zeros(-n)])


In [ ]:
info = calculate_everything(info,verbose=True)

In [ ]:
def plot_everything(D, look_direction=90, verbose=False):

#    pass
#if True:

    num_elements = len(D.Arr)
    num_sources = len(D.Src['absolute_direction'])
    
    li_left = np.argmin(np.abs(D.phi+look_direction))
    bs_left = D.phi[li_left] # beam steer
    li_right = np.argmin(np.abs(D.phi-look_direction))
    bs_right = D.phi[li_right] # beam steer
    
    if verbose:
        print(f'Look directions : {-look_direction}, {look_direction}')
        print(f'Look index : {li_left}, {li_right}')
        print(f'beam-steer : {bs_left:0.2f}, {bs_right:0.2f}') 
        print(f'beam-widths : {D.beam_widths[:,li_left]}, {D.beam_widths[:,li_right]}') 
        
    title_str = (f'Look directions : ({look_direction}°,' +
                   f'{-look_direction}°), ' +
                   f'Array angle : {D.array_angle}°, ' +
                   f'λ : {D.Src["wavelength"][0]:0.2f} m, ' +
                   f'f : {D.c/D.Src["wavelength"][0]:0.2f} Hz'
                  )
    
    #-------------------------------------------------------------------
    # Spacial Window
    #-------------------------------------------------------------------

    p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
            title=title_str)
    p.xaxis.axis_label = p.yaxis.axis_label = 'Distance (m)'
    p.x_range.range_padding = p.y_range.range_padding = 0

    # Plot the sound field
    D.spacial = p.image(image=[D.Z],
            x=D.x_min, y=D.x_min,
            dw=(D.x_max-D.x_min), dh=(D.x_max-D.x_min), 
            palette="Spectral11", level="image")

    # Add the look-directions to the plot
    ang = np.deg2rad(-look_direction + D.array_angle)
    x = [0,2*np.sin(ang)]
    y = [0,2*np.cos(ang)]
    D.look_beam_port = p.line(x,y,color='red',
        legend_label='Port Look Direction',line_width=2)
    ang = np.deg2rad(look_direction + D.array_angle)
    x = [0,2*np.sin(ang)]
    y = [0,2*np.cos(ang)]
    D.look_beam_starboard = p.line(x,y,color='green',
                legend_label='Starboard Look Direction', line_width=2)
    
    # Add the beamwidths to the plot
    ang1 = np.deg2rad(D.beam_widths[0,li_left] + D.array_angle)
    ang2 = np.deg2rad(D.beam_widths[1,li_left] + D.array_angle)
    x = [0,2*np.sin(ang1),2*np.sin(ang2)]
    y = [0,2*np.cos(ang1),2*np.cos(ang2)]
    D.beam_shade_port = p.patch(x,y,color='red',alpha=0.4,
        legend_label='Port Look Direction',
        line_width=0)
    ang1 = np.deg2rad(D.beam_widths[0,li_right] + D.array_angle)
    ang2 = np.deg2rad(D.beam_widths[1,li_right] + D.array_angle)
    x = [0,2*np.sin(ang1),2*np.sin(ang2)]
    y = [0,2*np.cos(ang1),2*np.cos(ang2)]
    D.beam_shade_starboard = p.patch(x,y,color='green',alpha=0.4,
        legend_label='Starboard Look Direction',line_width=0)

    # Add the source directions to the plot
    D.source_line = []
    for n in range(num_sources):
        D.source_line.append(p.line([0, D.Src['position'][n][0]],
                [0,D.Src['position'][n][1]],color='black',
                legend_label='Source direction(s)', line_width=2))
    
    # Add the sensors to the plot
    D.sensor = []
    for k in range(D.max_elements):
        D.sensor.append(p.circle([],[],size=10,fill_color=D.col[k],
                 legend_label=f'Sensor {k}',
                 ))
            
    for k in range(D.max_elements):
        if k < num_elements:
            D.sensor[k].visible = True
            x = [D.Arr[k][0]]
            y = [D.Arr[k][1]]
            D.sensor[k].data_source.data = {'x': x, 'y': y}
        else:
            D.sensor[k].visible = False

    D.spacial_legend = p.legend.items
    p.legend.items = D.spacial_legend[:3+num_elements]
    p.grid.grid_line_width = 0.5
    p.legend.click_policy="hide"
    p.legend.background_fill_alpha = 0.5
    p.x_range.start = D.x_min
    p.x_range.end = D.x_max
    p.y_range.start = D.x_min
    p.y_range.end = D.x_max
    D.p = p

    #-------------------------------------------------------------------
    # Time Series Window
    #-------------------------------------------------------------------

    q = figure(title=title_str)
        
    q.xaxis.axis_label = 'Time (s)'
    q.yaxis.axis_label = 'Pressure'
  
    # Plot the delay-sum for the look direction
    D.delay_sum = q.line(D.t,D.sum_F[li_right],color='black',
            legend_label='Theoretical Delay-sum')

    # Add the time series to the plot
    D.time_series = []
    for k in range(D.max_elements):
        D.time_series.append(q.line([],[],color=D.col[2*k],
                 legend_label=f'Sensor {k}',
                 ))
            
    # Plot the delay-sum for the look direction
    D.delay_sum_real = q.line(D.t,D.sum_F_real[li_right],color='cyan',
            legend_label='Real Delay-sum')

    # Add the time series to the plot
    D.time_series_real = []
    for k in range(D.max_elements):
        D.time_series_real.append(q.line([],[],color=D.col[2*k+1],
                 legend_label=f'Microphone {k}',
                 ))
            
    # plot the time series for each element
    plot_max = 0
    for k in range(D.max_elements):
        if k < num_elements:
            D.time_series[k].visible = True
            x = D.t
            y = pad(D.Y[k],-k*D.m[li_right])
            D.time_series[k].data_source.data = {'x': x, 'y': y}
            plot_max = np.max([plot_max,np.max(D.Y[k])])

            if D.real_data:
                D.time_series_real[k].visible = True
                x = D.t
                y = pad(D.W[k],-k*D.m[li_right])
                D.time_series_real[k].data_source.data = {'x': x, 'y': y}
                plot_max = np.max([plot_max,np.max(D.Y[k])])
            else:
                D.time_series_real[k].visible = False                
        else:
            D.time_series[k].visible = False
    
    # Add a shading for the region used to calculate the beams
    x = [D.t_0, D.t_0,D.t_0+D.sample_length,D.t_0+D.sample_length]
    
    y = plot_max*np.array([-1,1,1,-1])
    D.sample_patch = q.patch(x,y,color='yellow',alpha=0.2,line_width=0)
    
    D.temporal_legend = q.legend.items
    if D.real_data:
        q.legend.items = (D.temporal_legend[:2+num_elements] +
            D.temporal_legend[1+D.max_elements:2+D.max_elements+num_elements])
        D.delay_sum_real.visible = True
    else:
        q.legend.items = D.temporal_legend[:1+num_elements]        
        D.delay_sum_real.visible = False
    q.legend.click_policy="hide"
    q.legend.background_fill_alpha = 0.5
    D.q = q

    #-------------------------------------------------------------------
    # Beamforming Window
    #-------------------------------------------------------------------

    b = figure(title=title_str)
    
    b.xaxis.axis_label = 'Steering Angle (degrees)'
    b.yaxis.axis_label = 'Pressure'
    leg1 = 'Theoretical Beam Pattern'
    leg2 = 'Real Beam Pattern'
    
    x = D.phi
    y = D.beams
    D.beampattern_points = b.circle(x,y, legend_label=leg1)
    D.beampattern_line =  b.line(x,y, legend_label=leg1)
    
    D.look_line_port = Span(
        location=-look_direction,dimension='height',
        line_color='red', line_width=1)
    D.look_line_starboard = Span(
        location=look_direction,dimension='height',
        line_color='green', line_width=1)
    D.source_line = []
    for n in range(num_sources):
        D.source_line.append(Span(
            location=D.Src['absolute_direction'][n]-D.array_angle,
            dimension='height', line_color='black', line_width=2))
        
    b.line([[], []], legend_label='Port Look direction', 
           line_color="red", line_width=1)
    b.line([[], []], legend_label='Starboard Look direction', 
           line_color="green",line_width=1)
    b.line([[], []], legend_label='Source direction(s)', 
           line_color="black", line_width=2)
    b.renderers.extend([D.look_line_port,D.look_line_starboard, 
                        *D.source_line])

    # Add the beamwidths to the plot
    ang1 = D.beam_widths[0,li_left]
    ang2 = D.beam_widths[1,li_left]
    x = np.array([ang1, ang1 ,ang2, ang2])
    y = 1*np.array([0, np.max(D.beams),
         np.max(D.beams), 0])
    D.beampattern_shade_port = b.patch(x,y,color='red',alpha=0.4,
        legend_label='Port Look direction',line_width=0)
    ang1 = D.beam_widths[0,li_right]
    ang2 = D.beam_widths[1,li_right]
    x = np.array([ang1, ang1 ,ang2, ang2]) 
    y = 1*np.array([0, np.max(D.beams),
         np.max(D.beams), 0])
    D.beampattern_shade_starboard = b.patch(x,y,color='green',alpha=0.4,
        legend_label='Starboard Look direction',line_width=1)


    x = D.phi
    y = D.beams_real
    D.beampattern_points_real = b.circle(x,y, legend_label=leg2,
                                    color='green')
    D.beampattern_line_real =  b.line(x,y, legend_label=leg2,
                                 color='green')

    D.beamform_legend = b.legend.items
    if D.real_data:
        b.legend.items = D.beamform_legend
        D.beampattern_points_real.visible = True
        D.beampattern_line_real.visible = True
    else:
        b.legend.items = D.beamform_legend[:-1]        
        D.beampattern_points_real.visible = False
        D.beampattern_line_real.visible = False
    b.legend.background_fill_alpha = 0.5
    D.b = b
    
    return D

In [ ]:
info = plot_everything(info,verbose=True)

In [ ]:
show(gridplot([info.p, info.q, info.b], ncols=3), notebook_handle=True)

In [ ]:
def update(D,θ,look_direction,
           num_elements,
           Source_distance,wavelength,sample_length,t_0):
    
    num_sources = 1
    
    D = calculate_everything(D,
                             num_elements=num_elements,
                             array_angle=θ,
                             num_sources=num_sources,
                             src_radius=Source_distance,
                             wavelength=wavelength,
                             sample_length=sample_length,
                             t_0=t_0,
                            )

    li_left = np.argmin(np.abs(D.phi+look_direction))
    bs_left = D.phi[li_left] # beam steer
    li_right = np.argmin(np.abs(D.phi-look_direction))
    bs_right = D.phi[li_right] # beam steer
    
    
    title_str = (f'Look directions : ({look_direction}°,' +
                   f'{-look_direction}°), ' +
                   f'Array angle : {D.array_angle}°, ' +
                   f'λ : {D.Src["wavelength"][0]:0.2f} m, ' +
                   f'f : {D.c/D.Src["wavelength"][0]:0.2f} Hz'
                  )
    
    #-------------------------------------------------------------------
    # Update the spacial window
    #-------------------------------------------------------------------
    D.spacial.data_source.data['image'] = [D.Z]

    for k in range(D.max_elements):
        if k < num_elements:
            D.sensor[k].visible = True
            x = [D.Arr[k][0]]
            y = [D.Arr[k][1]]
            D.sensor[k].data_source.data = {'x': x, 'y': y}
        else:
            D.sensor[k].visible = False
    
    ang = np.deg2rad(-look_direction + D.array_angle)
    x = [0,2*np.sin(ang)]
    y = [0,2*np.cos(ang)]
    D.look_beam_port.data_source.data = {'x': x, 'y': y}
    ang = np.deg2rad(look_direction + D.array_angle)
    x = [0,2*np.sin(ang)]
    y = [0,2*np.cos(ang)]
    D.look_beam_starboard.data_source.data = {'x': x, 'y': y}

    # Update the beamwidths to the plot
    ang1 = np.deg2rad(D.beam_widths[0,li_left] + D.array_angle)
    ang2 = np.deg2rad(D.beam_widths[1,li_left] + D.array_angle)
    x = [0,2*np.sin(ang1),2*np.sin(ang2)]
    y = [0,2*np.cos(ang1),2*np.cos(ang2)]
    D.beam_shade_port.data_source.data = {'x': x, 'y': y}
    ang1 = np.deg2rad(D.beam_widths[0,li_right] + D.array_angle)
    ang2 = np.deg2rad(D.beam_widths[1,li_right] + D.array_angle)
    x = [0,2*np.sin(ang1),2*np.sin(ang2)]
    y = [0,2*np.cos(ang1),2*np.cos(ang2)]
    D.beam_shade_starboard.data_source.data = {'x': x, 'y': y}
    D.p.title.text = title_str
    D.p.legend.items = D.spacial_legend[:3+num_elements]
    
    #-------------------------------------------------------------------
    # Update the temporal window
    #-------------------------------------------------------------------
    plot_max = 0
    for k in range(D.max_elements):
        if k < num_elements:
            x = D.t
            D.time_series[k].visible = True
            if D.real_data:
                D.time_series_real[k].visible = True
            else:
                D.time_series_real[k].visible = False
            y = pad(D.Y[k],-D.m[li_right]*k)
            D.time_series[k].data_source.data = {'x': x, 'y': y}
            plot_max = np.max([plot_max,np.max(D.Y[k])])
            y = pad(D.W[k],-D.m[li_right]*k)
            D.time_series_real[k].data_source.data = {'x': x, 'y': y}
            plot_max = np.max([plot_max,np.max(D.W[k])])
        else:
            D.time_series[k].visible = False
            D.time_series_real[k].visible = False
    
    D.delay_sum.data_source.data = {'x': D.t,
                    'y': D.sum_F[li_right]}
    D.delay_sum_real.data_source.data = {'x': D.t,
                    'y': D.sum_F_real[li_right]}

    # Add a shading for the region used to calculate the beams
    x = [D.t_0, D.t_0,D.t_0+D.sample_length,D.t_0+D.sample_length]
    y = plot_max*np.array([-1,1,1,-1])
    D.sample_patch.data_source.data = {'x': x, 'y': y}

    D.q.title.text = title_str
    if D.real_data:
        D.q.legend.items = (D.temporal_legend[:1+num_elements] +
            D.temporal_legend[1+D.max_elements:2+D.max_elements+num_elements])
        D.delay_sum_real.visible = True
    else:
        D.q.legend.items = D.temporal_legend[:1+num_elements]        
        D.delay_sum_real.visible = False
        
    #-------------------------------------------------------------------
    # Update the Beamforming window
    #-------------------------------------------------------------------   
    x = D.phi
    y = D.beams
    D.beampattern_points.data_source.data = {'x': x, 'y': y}
    D.beampattern_line.data_source.data = {'x': x, 'y': y}
    y = D.beams_real
    D.beampattern_points_real.data_source.data = {'x': x, 'y': y}
    D.beampattern_line_real.data_source.data = {'x': x, 'y': y}
    D.look_line_port.location = -look_direction
    D.look_line_starboard.location = look_direction

    # Add the beamwidths to the plot
    ang1 = D.beam_widths[0,li_left]
    ang2 = D.beam_widths[1,li_left]
    x = np.array([ang1, ang1 ,ang2, ang2])
    y = 1*np.array([0, np.max(D.beams),
         np.max(D.beams), 0])
    D.beampattern_shade_port.data_source.data = {'x': x, 'y':y}
    ang1 = D.beam_widths[0,li_right]
    ang2 = D.beam_widths[1,li_right]
    x = np.array([ang1, ang1 ,ang2, ang2]) 
    y = 1*np.array([0, np.max(D.beams),
         np.max(D.beams), 0])
    D.beampattern_shade_starboard.data_source.data = {'x': x, 'y':y}

    for n in range(num_sources):
        source_direction = (D.Src['absolute_direction'][n] - D.array_angle)
        if source_direction > 180:
            source_direction = source_direction - 360
        if source_direction < -180:
            source_direction = source_direction + 360
        D.source_line[n].location = source_direction

    D.b.title.text = title_str
    if D.real_data:
        D.b.legend.items = D.beamform_legend
        D.beampattern_points_real.visible = True
        D.beampattern_line_real.visible = True
    else:
        D.b.legend.items = D.beamform_legend[:-1]        
        D.beampattern_points_real.visible = False
        D.beampattern_line_real.visible = False

    push_notebook()


In [ ]:
w = interactive(update, D=fixed(info), 
         θ=(-180, 180, 1),
         look_direction = (1,179,1),
         num_elements= (2,12,1),
         Source_distance=(1,100), 
         wavelength=(0.01,0.4,0.01),
         sample_length=(0.001,0.01,0.001),
         t_0=(0,0.01,0.0001),
        )

for q in w.children:
    q.style = {'description_width': 'initial'}

grid = GridspecLayout(2, 4)

for i in range(2):
    for j in range(4):
        if i*4+j <= 7:
            grid[i, j] = w.children[i*4+j]

In [ ]:
show(gridplot([info.p, info.q, info.b], ncols=3), notebook_handle=True)
grid[1,0].value = 0.21
grid
